# AIDM

This is an introduction to the ai_dm library, which is meant to provide an introduction to the basic approaches to AI decisions making using state space search and planning, RL, MCTS and more.


The following command installs our library

In [2]:
!pip install git+https://github.com/CLAIR-LAB-TECHNION/aidm


  Cloning https://github.com/CLAIR-LAB-TECHNION/aidm to /tmp/pip-req-build-h43hfrhy
  Running command git clone --filter=blob:none --quiet https://github.com/CLAIR-LAB-TECHNION/aidm /tmp/pip-req-build-h43hfrhy
  Resolved https://github.com/CLAIR-LAB-TECHNION/aidm to commit f6ae8bebf89b461cdea52769defd051649e79984
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for aidm: filename=aidm-0.0.0-py3-none-any.whl size=25446 sha256=030fc75b8055f5aa362784edb6ed52984d55ad1b1e25ac2bbda5be8667324799
  Stored in directory: /tmp/pip-ephem-wheel-cache-qxq14r_v/wheels/a0/b2/49/c9435c200208d7986c815b4e2dee7b96d2f747ec15de418163
Successfully built aidm


And this command installs OpenAI's gym library which offers a diverse collection of environments to work with

In [3]:
!pip install "gym>=0.26"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.7/721.7 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for gym: filename=gym-0.26.2-py3-none-any.whl size=827621 sha256=b83d3f8fe36a7a1c2214ee0f6fc38e3214fe10aa272e3fb12ab65614d125180c
  Stored in directory: /root/.cache/pip/wheels/b9/22/6d/3e7b32d98451b4cd9d12417052affbeeeea012955d437da1da
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
dopamine-rl 4.0.6 requires gym<=0.25.2, but you have gym 0.26.2 which is incompatible.


We will be working with Open Ai's Taxi domain that looks like this


In [4]:
import gym

env = gym.make("Taxi-v3", render_mode='ansi').env
env.reset()
print(env.render())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




The filled square represents the taxi, which is yellow without a passenger and green with a passenger. The pipes ("|") represent walls which the taxi cannot traverse. R, G, Y, B are the possible pickup and destination locations. The passenger can also be in the taxi. The blue letter represents the current passenger pick-up location, and the purple letter is the current destination.

The actions space is:

0 = south 1 = north 2 = east 3 = west 4 = pickup 5 = dropoff

The state encoding is as follows (play around with the values)

In [5]:
state = env.encode(1, 3, 4, 1) # (taxi row, taxi column, passenger index, destination index)
print("State:", state)

print(env.render())
env.unwrapped.s = state
print(env.render())

State: 177
+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




Now let's import the GymProblem which will wrap the Taxi domain

In [7]:
import gym
from aidm.environments.gym_envs.gym_problem import GymProblem

and now let's import the search methods we want to apply


In [8]:

from aidm.search.best_first_search import best_first_search, breadth_first_search, depth_first_search, a_star, depth_first_search_l
import aidm.search.utils as utils
import aidm.search.defs as defs
import aidm.search.heuristic as heuristic




The following will create and recreate the taxi environment we want to explore

In [9]:
def create_env():

    # define the environment
    taxi_env = gym.make("Taxi-v3", render_mode='ansi').env
    taxi_env.reset()
    init_state = taxi_env.encode(0, 3, 4, 1)  # (taxi row, taxi column, passenger index, destination index)
    taxi_row, taxi_col, pass_idx, dest_idx = taxi_env.decode(init_state)
    print(taxi_row)
    taxi_env.unwrapped.s = init_state
    print("State:", init_state)
    print(env.render())
    return taxi_env


And now let's run Breadth First Search


In [10]:
taxi_env = create_env()

# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)


# perform BFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = breadth_first_search(problem=taxi_p,
                                                                                         log=True,
                                                                                         log_file=None,
                                                                                         iter_limit=defs.NA,
                                                                                         time_limit=defs.NA,
                                                                                        )


print(best_plan)
for action_id in best_plan:
    print(taxi_env.step(int(action_id)))
    taxi_p.apply_action(action_id)
    print(taxi_p.env.render())


0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 177>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 177>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 5 cur_node:<Node 277>
InMethod best_first_design(node): explored_count:5 cur_node:<Node 277>, node_eval_time:0.000
best_first_design(node) node number 6 cur_node:<Node 197>
InMethod best_first_design(node): explored_count:6 cur_node:<Node 197>, node_eval_time:0.000
be

/usr/local/lib/python3.10/dist-packages/gym/utils/passive_env_checker.py:233: DeprecationWarning: `np.bool8` is a deprecated alias for `np.bool_`.  (Deprecated NumPy 1.24)
  if not isinstance(terminated, (bool, np.bool8)):


Let's see the same methods, but now with the explicit call to BestFirstSearch (which shows us how Breadth First Search is a special case of BestFirstSearch)

In [11]:
taxi_env = create_env()

# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform BFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = best_first_search(problem=taxi_p,
                                                                                      frontier=utils.FIFOQueue(),
                                                                                      closed_list=utils.ClosedListOfKeys(),
                                                                                      termination_criteria=utils.TerminationCriteriaGoalStateReached(),
                                                                                      evaluation_criteria=utils.EvaluationCriteriaGoalCondition(),
                                                                                      prune_func=None,
                                                                                      log=True, log_file=None,
                                                                                      iter_limit=defs.NA,
                                                                                      time_limit=defs.NA,
                                                                                      )
print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    print(env.render())

0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 177>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 177>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 5 cur_node:<Node 277>
InMethod best_first_design(node): explored_count:5 cur_node:<Node 277>, node_eval_time:0.000
best_first_design(node) node number 6 cur_node:<Node 197>
InMethod best_first_design(node): explored_count:6 cur_node:<Node 197>, node_eval_time:0.000
be

**And** now, let's run DFS

In [12]:
taxi_env = create_env()
# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform DFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = depth_first_search(problem=taxi_p,
                                                                                       log=True,
                                                                                       log_file=None,
                                                                                       iter_limit=defs.NA,
                                                                                       time_limit=defs.NA,
                                                                                       )

print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    print(env.render())

0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 157>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 157>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 257>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 257>, node_eval_time:0.000
best_first_design(node) node number 5 cur_node:<Node 237>
InMethod best_first_design(node): explored_count:5 cur_node:<Node 237>, node_eval_time:0.000
best_first_design(node) node number 6 cur_node:<Node 217>
InMethod best_first_design(node): explored_count:6 cur_node:<Node 217>, node_eval_time:0.000


and now let's run DFS-L

In [13]:
taxi_env = create_env()
# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform DFS
[best_value, best_node, best_plan, explored_count, ex_terminated] = depth_first_search_l(problem=taxi_p,
                                                                                       max_depth=2,
                                                                                       log=True,
                                                                                       log_file=None,
                                                                                       iter_limit=defs.NA,
                                                                                       time_limit=defs.NA,
                                                                                       )

print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    print(env.render())

0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 177>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 177>, node_eval_time:0.000
solution is: []
[]


Finally, let's see how A-Star works on the same problem (with a ridiculously simple heuristic that gives 0 to terminal nodes and 1 to all other nodes):

In [14]:
taxi_env = create_env()

# create a wrapper of the environment to the search
taxi_p = GymProblem(taxi_env, taxi_env.unwrapped.s)

# perform A*
[best_value, best_node, best_plan, explored_count, ex_terminated] = best_first_search(problem=taxi_p,
                                                                                      frontier=utils.PriorityQueue(heuristic.goal_heuristic),
                                                                                      closed_list=utils.ClosedListOfKeys(),
                                                                                      termination_criteria=utils.TerminationCriteriaGoalStateReached(),
                                                                                      evaluation_criteria=utils.EvaluationCriteriaGoalCondition(),
                                                                                      prune_func=None,
                                                                                      log=True,
                                                                                      log_file=None,
                                                                                      iter_limit=defs.NA,
                                                                                      time_limit=defs.NA,
                                                                                      )

print(best_plan)
for action_id in best_plan:
    taxi_p.apply_action(action_id)
    print(env.render())

0
State: 77
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


best_first_design(node) node number 1 cur_node:<Node 77>
InMethod best_first_design(node): explored_count:1 cur_node:<Node 77>, node_eval_time:0.000
best_first_design(node) node number 2 cur_node:<Node 57>
InMethod best_first_design(node): explored_count:2 cur_node:<Node 57>, node_eval_time:0.000
best_first_design(node) node number 3 cur_node:<Node 97>
InMethod best_first_design(node): explored_count:3 cur_node:<Node 97>, node_eval_time:0.000
best_first_design(node) node number 4 cur_node:<Node 85>
InMethod best_first_design(node): explored_count:4 cur_node:<Node 85>, node_eval_time:0.000
solution is: ['2', '5']
['2', '5']
+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+


+---------+
|R: | : :G|
| : | :_: |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+




**RL**

Another approach to solve the taxi domain is to apply a reinforcement learning (RL) approach by which agents learn by interacting with the environment.  

First, let's see what would happen if we try to brute-force our way to solving the problem.

Since we have our P table for default rewards in each state, we can try to have our taxi navigate just using that.

We'll create an infinite loop which runs until one passenger reaches one destination (one episode), or in other words, when the received reward is 20. The env.action_space.sample() method automatically selects one random action from set of all possible actions.

In [15]:
env.s = 328  # set environment to illustration's state

epochs = 0
penalties, reward = 0, 0

frames = [] # for animation

done = False

while not done:
    action = env.action_space.sample()
    out = env.step(action)
    # print(out)

    state, reward, done, trunc, info = out

    if reward == -10:
        penalties += 1

    # Put each rendered frame into dict for animation
    frames.append({
        'frame': env.render(),
        'state': state,
        'action': action,
        'reward': reward
        }
    )

    epochs += 1


print("Timesteps taken: {}".format(epochs))
print("Penalties incurred: {}".format(penalties))

Timesteps taken: 108
Penalties incurred: 40


We can now play the animation of the behaviors (and stop it when we want to by pressing the stop button)



In [16]:
from IPython.display import clear_output
from time import sleep

def print_frames(frames):
    for i, frame in enumerate(frames):
        clear_output(wait=True)
        print(frame['frame'])
        print(f"Timestep: {i + 1}")
        print(f"State: {frame['state']}")
        print(f"Action: {frame['action']}")
        print(f"Reward: {frame['reward']}")
        sleep(.1)

print_frames(frames)

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (Dropoff)

Timestep: 108
State: 85
Action: 5
Reward: 20


**Q Learning**


Essentially, Q-learning lets the agent use the environment's rewards to learn over time the best action to take in a given state.

In our Taxi environment, we have the reward table, P, that the agent will learn from. It does this by receiving a reward for taking an action in the current state, then updating a Q-value to remember if that action was beneficial.

The values store in the Q-table are called a Q-values, and they map to a (state, action) combination.

A Q-value for a particular state-action combination is representative of the "quality" of an action taken from that state. Better Q-values imply better chances of getting greater rewards.

For example, if the taxi is faced with a state that includes a passenger at its current location, it is highly likely that the Q-value for pickup is higher when compared to other actions, like dropoff or north.

Q-values are initialized to an arbitrary value, and as the agent exposes itself to the environment and receives different rewards by executing different actions, the Q-values are updated using the equation:

Q(state,action)←(1−α)Q(state,action)+α(reward+γmaxaQ(next state,all actions)) Where:

α (alpha) is the learning rate (0<α≤1) - Just like in supervised learning settings, α is the extent to which our Q-values are being updated in every iteration.

γ (gamma) is the discount factor (0≤γ≤1) - determines how much importance we want to give to future rewards. A high value for the discount factor (close to 1) captures the long-term effective award, whereas, a discount factor of 0 makes our agent consider only immediate reward, hence making it greedy.

In [17]:
print(env.observation_space)

Discrete(500)


In [18]:
from aidm.rl.q_learning import q_learning


# perform q_learning
#[best_value, best_node, best_plan, explored_count, ex_terminated] = \
q_learning(problem=taxi_p,learning_rate=0.9, discount_rate=0.8, epsilon=1.0, decay_rate=0.005, num_episodes=1000,
                   max_steps_per_episode=99,  num_of_trials=100, log=False, log_file=None)

Training completed over 1000 episodes


array([[  0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ],
       [ -3.7802152 ,  -2.45943816,  -4.03964283,  -3.80019289,
         -1.6445568 , -11.257668  ],
       [ -0.48859388,  -3.19134889,  -3.53003392,  -3.18699174,
          3.192     ,  -7.5235544 ],
       ...,
       [ -2.88870028,  -2.88543062,  -2.93374207,  -1.15447999,
        -11.3540976 , -12.25359726],
       [ -3.26994459,  -3.29490829,  -3.33300639,  -1.04919711,
        -10.797588  , -12.22486457],
       [ -1.638     ,  -0.99      ,  -0.99      ,  14.99840998,
         -9.        ,  -9.        ]])